In [106]:
import pandas as pd
import sqlalchemy as sa
from redshift_credentials import database, username, password, host, port, schema
from sqlalchemy.orm import sessionmaker
import itertools

In [5]:
def sql_con(in_username, in_password, in_host, in_port, in_database, in_schema):
    '''Create connection function'''
    connection = "redshift+psycopg2://%s:%s@%s:%s/%s" % (in_username, in_password, in_host, str(in_port), in_database)
    engine = sa.create_engine(connection)
    session = sessionmaker()
    session.configure(bind=engine)
    s = session()
    setpath = "SET search_path TO %s" % in_schema
    s.execute(setpath)
    return engine

In [7]:
# Define connection.
conn = sql_con(username, password, host, port, database, schema)

In [230]:
# Primary forecast query.
forecast_query = "SELECT fulfillment_week_ended week_ended, %s, %s qty_returned, COUNT(*) qty_sold, %s * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1%s"

# Actual primary query.
actual_query = "SELECT fulfillment_week_ended week_ended, %s, %s qty_returned, COUNT(*) qty_sold, %s * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1%s"

In [231]:
# Forecasted qty returned/exchanged query.
forecast_qty = "SUM(CASE WHEN send_back__type IN %s AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END)"

# Actual qty returned/exchanged query.
actual_qty = "SUM(CASE WHEN send_back__type in %s THEN 1 ELSE 0 END)"

In [232]:
product_categories = "CASE WHEN shape = 'Rectangle' AND size NOT ILIKE ('%9%x%12%') AND texture = 'Chenile' AND purpose != 'Outdoor' THEN 'rectangular_chenille' WHEN texture = 'Shag' THEN 'shag' WHEN shape = 'Round' THEN 'round' WHEN purpose = 'Outdoor' THEN 'outdoor' WHEN texture = 'Berber' THEN 'plush' WHEN size ILIKE ('%9%x%12%') THEN '9x12' WHEN texture = 'Rubber' THEN 'rug_pad' END"

product_cushioned = "CASE WHEN product_sub_type = 'cushioned' THEN 'cushioned' ELSE 'classic' END"

In [233]:
query_params = {
    'return_product_weekly': {'view': ['actual', 'forecast'],
                              'product': ['all', 'categories', 'cushioned'],
                              'send_back__type': [('return', 'exchange'), ('return'), ('exchange')]}
}

In [235]:
for key, value in query_params.items():
    '''Add logic to populate each tab in google sheet'''
    combinations = []
    keys = value.keys()
    values = (value[key] for key in keys)
    combinations.append(dict(zip(keys, combination)) for combination in itertools.product(*values))
    for combination in combinations:
        for query in combination:
            keys = list(keys)
            grouping = ""
            if query[keys[0]] == 'actual':
                for i in range(len(keys)):
                    if i == 0:
                        params = f"'{query[keys[i]]}' AS {keys[i]}"
                    if keys[i] == 'product':
                        if query[keys[i]] == 'all':
                            params = f"{params}, '{query[keys[i]]}' AS {keys[i]}"
                        elif query[keys[i]] == 'categories':
                            params = f"{params}, {product_categories} AS {keys[i]}"
                        else:
                            params = f"{params}, {product_cushioned} AS {keys[i]}"
                    if keys[i] == 'send_back__type':
                        if len(query[keys[i]]) == 2:
                            params = f"{params}, 'send_back_both' AS {keys[i]}"
                            actual_qty_updated = actual_qty %(str(query[keys[i]]))
                        else:
                            params = f"{params}, {query[keys[i]]} AS {keys[i]}"
                            actual_qty_updated = actual_qty %(f"({query[keys[i]]})")
                    if i > 0:
                        grouping = f"{grouping}, {i+1}"
                print(actual_query %(params, actual_qty_updated, actual_qty_updated, grouping))

SELECT fulfillment_week_ended week_ended, 'actual' AS view, 'all' AS product, 'send_back_both' AS send_back__type, SUM(CASE WHEN send_back__type in ('return', 'exchange') THEN 1 ELSE 0 END) qty_returned, COUNT(*) qty_sold, SUM(CASE WHEN send_back__type in ('return', 'exchange') THEN 1 ELSE 0 END) * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1, 2, 3
SELECT fulfillment_week_ended week_ended, 'actual' AS view, 'all' AS product, return AS send_back__type, SUM(CASE WHEN send_back__type in (return) THEN 1 ELSE 0 END) qty_returned, COUNT(*) qty_sold, SUM(CASE WHEN send_back__type in (return) THEN 1 ELSE 0 END) * 1.0 / COUNT(*) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1, 2, 3
SELECT fulfillment_week_ended week_ended, 'actual' AS view, 'all' AS product, exchange AS send_back__type, SUM(CASE WHEN send_back__type in (exchange) THEN 1 ELSE 0 END) qty_returned, COUNT(*) qty_sold, SUM(CASE WHEN send_back__type in (exchange) THEN 1 ELSE 0 END) * 1.0 / COUNT(

In [105]:
for key, value in query_params.items():
    '''Add logic to populate each tab in google sheet'''
    for key1, value1 in value.items():
#         print(key1, end=": ")
        for item in value1:
#             print(item)
            

view: actual
forecast
product: all
categories
cushioned
send_back_type: ('return', 'exchange')
return
exchange


In [12]:
df = pd.read_sql(f"SELECT {c} FROM product_pull.ret_exch_rep_data_final LIMIT 10", conn)

In [13]:
df.head()

,email,cancelled_at,order_number,order_name,created_at_time,created_date,week_ended,month_ended,gross_sales,discount,...,price,total,loop_refund,label_updated_at,label_status,state,tracking_number,returned_at,total_items_exchanged_for,row_num_item_loop
0,msjaime81@gmail.com,None,1000133,None,2020-07-27 14:55:43,2020-07-27,2020-08-02,2020-07-31,199.0,29.85,...,None,None,None,None,None,None,None,None,None,None
1,emilypatel.623@gmail.com,None,100145,None,2019-02-03 20:50:51,2019-02-03,2019-02-03,2019-02-28,139.0,20.85,...,None,None,None,None,None,None,None,None,None,None
2,lindsey.a.cambridge@gmail.com,None,1001801,None,2020-07-27 17:37:28,2020-07-27,2020-08-02,2020-07-31,149.0,22.35,...,None,None,None,None,None,None,None,None,None,None
3,canam711@yahoo.com,None,1004335,None,2020-07-27 20:55:08,2020-07-27,2020-08-02,2020-07-31,399.0,59.85,...,None,None,None,None,None,None,None,None,None,None
4,kafrenkiel@gmail.com,None,1002987,None,2020-07-27 19:08:54,2020-07-27,2020-08-02,2020-07-31,149.0,22.35,...,None,None,None,None,None,None,None,None,None,None
